In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import re
import os
from openpyxl import load_workbook
from shutil import move
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait

In [8]:
url="https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page={}&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_{}"
options = webdriver.ChromeOptions()
options.add_argument("--disable-notifications")
options.add_argument('headless')
# options.add_argument('Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like Mac OS X) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/13.0 MQQBrowser/10.1.1 Mobile/15B87 Safari/604.1 QBWebViewUA/2 QBWebViewType/1 WKType/1')
chrome = webdriver.Chrome('./chromedriver')
data = []
tStart = time.time()
for i in range(1,3):
    chrome.get(url.format(i,i))
    chrome.find_element_by_id('twotabsearchtextbox').send_keys(Keys.F12)
    print(url.format(i,i))
#     time.sleep(5)
    soup = BeautifulSoup(chrome.page_source,'lxml')
    amazon_card=soup.find_all(class_="a-size-medium a-color-base a-text-normal")
    if len(amazon_card) == 0:
        print("找不到顯示卡標題")
        break
    data.append([ele.text for ele in amazon_card])   
chrome.close()    
tEnd = time.time()
print("總共耗時:%f 秒" % (tEnd-tStart))

https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=1&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_1
https://www.amazon.com/-/zh_TW/s?i=electronics-intl-ship&bbn=16225009011&rh=n%3A541966%2Cn%3A193870011%2Cn%3A17923671011%2Cn%3A284822&dc&page=2&language=zh_TW&qid=1598425548&rnid=17923671011&ref=sr_pg_2
總共耗時:10.045718 秒


In [9]:
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
print(amazon_json['title'])

['EVGA', 'ZOTAC', 'ASUS', 'NVIDIA', 'MSI微星', 'XFX', 'Gigabyte', 'MSI', 'PowerColor', 'Sapphire']


In [10]:
title=[]
for adx,datas in enumerate(data):
    
    df = 'df'+str(adx) 
    df = pd.DataFrame(datas,columns=['f'])
    columns_of_datas = [ddd for ddd in range(0,len(datas))]  
    # 分種類並寫入json中
    for aaa in columns_of_datas:
    # 正則表達式將標題的牌子切割出來
        matchObj = re.split(r'\s', df['f'][aaa], re.M|re.I)
        title.append(matchObj[0])
# 篩選重複的牌子        
json_title_dumps = list(set(title))
json_title = {'title': json_title_dumps}
amazon_json.update(json_title)
# 將整理好title的json寫入json檔案中
with open('AMAZON_COMPUTER_PRICE.json', 'w', encoding='utf-8') as f:
    json.dump(amazon_json, f, ensure_ascii=False, indent=4,sort_keys=True)
print(amazon_json)

{'sheet_name': '顯示卡', 'title': ['PowerColor', 'EVGA', 'Gigabyte', 'Sapphire', 'MSI微星', 'NVIDIA', 'MSI', 'ASUS', 'ZOTAC', 'XFX']}


In [21]:
now_date = datetime.strftime(datetime.now() , '%Y%m%d')
amazon_json = json.load(open("AMAZON_COMPUTER_PRICE" + ".json",encoding="utf-8"))
# 沒檔案的話則建立
output_file = now_date + "_amazon_graphics card.xlsx"
target_path = "D:\\Amazon_graphics_card\\"

writer = pd.ExcelWriter(output_file, engine='openpyxl')
# 20200924  
title_dict={}
def title_var(title,datas):
    
    for s_datas in datas:
        for s_title in title:
            if s_title in s_datas: 
                title_dict.setdefault(s_title,[]).append(s_datas)
    return title_dict


for adx,datas in enumerate(data):   
    title_dict = title_var(amazon_json['title'],datas)   

df = pd.DataFrame(title_dict.items()).T
df.to_excel(writer,sheet_name=amazon_json['sheet_name']) 
writer.save() 

In [22]:
cwd = os.getcwd()
curr_file = cwd + "\\" + output_file
target_file = target_path + output_file

try:
    move(curr_file, target_file)
    print(output_file + "\n移動檔案完畢.")
except Exception as e:
    print(output_file + "\n移動檔案失敗.")
    print(e.args)

20200924_amazon_graphics card.xlsx
移動檔案完畢.


In [23]:
title_dict

{'XFX': ['XFX HDMI 顯卡',
  'XFX HDMI 顯卡',
  'XFX RX 5700 Xt Thicc III Ultra 8GB Boost 高達 2025MHz GDDR6 3xDP HDMI 顯示卡(Rx-57XT8TBD8)',
  'XFX RX 5700 XT 三重耗 8GB GDDR6 3xDP HDMI 顯示卡 RX-57XT83LD8'],
 'EVGA': ['EVGA 06G-P4-2066-KR GeForce RTX 2060 KO Gaming,6GB GDDR6,雙風扇,金屬背板',
  'EVGA 06G-P4-1068-KR GeForce GTX 1660 Super Sc Ultra Gaming, 6GB GDDR6, Dual Fan, Metal Backplate',
  'EVGA 08G-P4-3173-KR GeForce RTX 2070 Super XC ultra gaming,8GB GDDR6,雙 HDB 風扇,RGB LED,金屬背板',
  'EVGA GeForce RTX 2070 Super KO Gaming,08G-P4-2072-KR,8GB GDDR6,雙風扇',
  'EVGA 08G-P4-3071-KR GeForce RTX 2070 Super Gaming, 8GB GDDR6 處理器',
  'EVGA GeForce GTX 1650 Super SC Ultra Gaming, 4GB GDDR6, Dual Fan, Metal Backplate, 04G-P4-1357-KR PC（聚碳酸酯）',
  'EVGA GeForce RTX 2070 Super XC 電競,8GB GDDR6,雙 HDB 風扇,RGB LED,金屬背板,08G-P4-3172-KR'],
 'MSI微星': ['MSI微星 Gaming GeForce RTX 2060 6GB GDRR6 192 位元 HDMI/DP 光線追蹤圖靈架構VR Ready 顯卡(RTX 2060 VENTUS XS 6G OC)',
  'MSI微星 Gaming GeForce RTX 2060 6GB GDRR6 192 位元 HDMI/DP 光線追蹤圖層架構VR Read